In [3]:
from sqlalchemy import create_engine
import pandas as pd

In [4]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [5]:
engine.table_names()

['Bookings', 'Facilities', 'Members']

In [46]:
facilities = pd.read_sql_query('select * from Facilities', engine)
facilities.head(1)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200


In [47]:
bookings = pd.read_sql_query('select * from Bookings', engine)
bookings.head(1)

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2


In [48]:
members = pd.read_sql_query('select * from Members', engine)
members.head(1)

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00


In [49]:
bookings_per_member_facility = pd.read_sql_query('select memid,facid,count(distinct bookid) as bookings from Bookings group by memid, facid', engine)
bookings_per_member_facility.head(3)

,memid,facid,bookings
0,0,0,100
1,0,1,113
2,0,2,39


In [50]:
print(bookings_per_member_facility.shape)
df = bookings_per_member_facility.merge(facilities, how = 'left', on = 'facid')
print(df.shape)
#df.head()

(211, 3)
(211, 8)


In [51]:
#df.tail()

In [52]:
# calculating revenue column
df['revenue'] = (df['memid'] == 0)*df['guestcost']*df['bookings'] + (df['memid'] != 0)*df['membercost']*df['bookings']
df.head(1)

,memid,facid,bookings,name,membercost,guestcost,initialoutlay,monthlymaintenance,revenue
0,0,0,100,Tennis Court 1,5.0,25.0,10000,200,2500.0


In [35]:
df1 = pd.DataFrame(df)
df1.head(1)

,memid,facid,bookings,name,membercost,guestcost,initialoutlay,monthlymaintenance,revenue
0,0,0,100,Tennis Court 1,5.0,25.0,10000,200,2500.0


In [36]:
# dropping unneeded columns
del df1['memid']
del df1['facid']
del df1['bookings']
del df1['membercost']
del df1['guestcost']
del df1['initialoutlay']
del df1['monthlymaintenance']

df1.head()

,name,revenue
0,Tennis Court 1,2500.0
1,Tennis Court 2,2825.0
2,Badminton Court,604.5
3,Table Tennis,90.0
4,Massage Room 1,16640.0


In [40]:
df2 = (df1.groupby(by='name').sum().reset_index())
print(df1.shape)
print(df2.shape)

(211, 2)
(9, 2)


# <font color = 'blue'> Question 10 answer below:</font>

In [45]:
df2.loc[df2.revenue < 1000].sort_values(by = 'revenue').reset_index(drop=True)

,name,revenue
0,Table Tennis,90.0
1,Snooker Table,115.0
2,Pool Table,265.0
3,Badminton Court,604.5


=============================================
=============================================

In [65]:
#members.head()

# <font color = 'blue'> Question 11 answer below:</font>

In [66]:
members_recommenders = pd.read_sql_query("select (Members.firstname||' '||Members.surname) as name, (b.firstname||' '||b.surname) as recommendedby_name  from Members left join(select memid,firstname,surname from Members) as b on Members.recommendedby = b.memid", engine)
members_recommenders

,name,recommendedby_name
0,GUEST GUEST,None
1,Darren Smith,None
2,Tracy Smith,None
3,Tim Rownam,None
4,Janice Joplette,Darren Smith
5,Gerald Butters,Darren Smith
6,Burton Tracy,None
7,Nancy Dare,Janice Joplette
8,Tim Boothe,Tim Rownam
9,Ponder Stibbons,Burton Tracy


=============================================
=============================================

In [76]:
bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


# <font color = 'blue'> Question 12 answer below:</font>

In [75]:
facil_usage_by_members = pd.read_sql_query('with f as (select facid, name from Facilities group by facid,name) select f.name, count(distinct a.memid) as members from Bookings as a left join f on a.facid = f.facid where a.memid != 0 group by f.name',engine)
facil_usage_by_members

,name,members
0,Badminton Court,24
1,Massage Room 1,24
2,Massage Room 2,12
3,Pool Table,27
4,Snooker Table,22
5,Squash Court,24
6,Table Tennis,25
7,Tennis Court 1,23
8,Tennis Court 2,21


=============================================
=============================================

In [99]:
from sqlalchemy.sql import func
[func.substr(bookings.starttime, 6, 2)=='07']

In [91]:
from sqlalchemy.sql import func

pd.read_sql_query('with f as (select facid, name from Facilities group by facid,name), select f.name, extract(date from starttime) as month, count(distinct a.memid) as members from Bookings as a left join f on a.facid = f.facid where a.memid != 0 group by f.name, extract(date from starttime)',engine)

OperationalError: (sqlite3.OperationalError) near "from": syntax error
[SQL: with f as (select facid, name from Facilities group by facid,name) select f.name, extract(date from starttime) as month, count(distinct a.memid) as members from Bookings as a left join f on a.facid = f.facid where a.memid != 0 group by f.name, extract(date from starttime)]
(Background on this error at: http://sqlalche.me/e/13/e3q8)